In [ ]:
#Best-fit Model Lightcurve 
def batman_curve(time, rp, a, inc, t0, per, ecc, w, u=[0.3,0.2], limb_dark="quadratic"):
    params = batman.TransitParams()
    params.t0 = t0
    params.per = per
    params.rp = rp
    params.a = a
    params.inc = inc
    params.ecc = ecc
    params.w = w
    params.u = u
    params.limb_dark = limb_dark
    m = batman.TransitModel(params, time)
    return m.light_curve(params)

model_best = batman_curve(t, rp, a, t0, inc, per, ecc, w)
print(t)

plt.figure(figsize=(8,5))
plt.errorbar(t, flux, yerr=flux_err, fmt=".k", ms=4, label="data", alpha=0.6)
plt.plot(t, model_best, color="C1", lw=2, label="best-fit model")
plt.xlabel("Time [days]")
plt.ylabel("Relative flux")
plt.legend()
plt.title("Transit fit with batman + emcee")
plt.show()

In [ ]:
#Best-fit Orbital Model
a_Rs = 19.49 # Semi-major axis (a/Rs)
eccentricity = 0.14090   # Eccentricity (e)
inc_deg = 80.96      # Inclination (i)
omega_deg = 184.9    # Argument of Periapsis (omega)

# Conversions
a_physical = a_Rs * 1.0 # Plotting scale is 1 R_s = 1 unit
inc_rad = np.radians(inc_deg)
omega_rad = np.radians(omega_deg)

def plot_3d_eccentric_orbit(a, e, inc_rad, omega_rad):
    """
    Plots the 3D orbit of a highly eccentric body (like HD 80606b).
    """
    # Create the True Anomaly angle (v) for 360 degrees of orbit
    v = np.linspace(0, 2 * np.pi, 500)

    # 1. Calculate the instantaneous radial distance r(v)
    # r(v) = [a * (1 - e^2)] / [1 + e * cos(v)]
    numerator = a * (1.0 - e**2)
    denominator = 1.0 + e * np.cos(v)
    r_v = numerator / denominator
   
    # 2. Calculate the angular position in the orbital plane (angle from star to planet)
    true_angle = omega_rad + v # Angle from X-axis, rotated by omega

    # 3. Parametric Equations for an Inclined Elliptical Orbit (Simplified Frame):
    X = r_v * np.cos(true_angle)
    Y = r_v * np.sin(true_angle) * np.cos(inc_rad)
    Z = r_v * np.sin(true_angle) * np.sin(inc_rad)

    # --- Setup Plot ---
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111, projection='3d')
   
    # 1. Plot the Orbit (Blue Line)
    ax.plot(X, Y, Z, color='blue', label=f'HD 80606b Orbit (e={e}, i={np.degrees(inc_rad):.2f}°)')
   
    # 2. Highlight Apoapsis and Periapsis
    ax.scatter(X[0], Y[0], Z[0], color='red', s=10, label='Periapsis (Closest)')
    ax.scatter(X[250], Y[250], Z[250], color='green', s=10, label='Apoapsis (Farthest)')

    # 3. Plot the Host Star (HD 80606)
    ax.scatter(0, 0, 0, color='gold', marker='o', s=100, label='Host Star HD 80606')

    ax.set_xlim([-30, 30])
    ax.set_ylim([-30, 30])
    ax.set_zlim([-30, 30])
   
    ax.set_xlabel('X (A/Rs)')
    ax.set_ylabel('Y (B/Rs - Plane of Sky)')
    ax.set_zlabel('Z (C/Rs - Line of Sight Depth)')
    ax.set_title(f'3D Eccentric Orbit of HD 80606b (a/Rs = {a:.2f}, e = {e})')

    ax.legend(loc='upper right')
    plt.show()

# --- Run the Plotting Code ---
plot_3d_eccentric_orbit(a_physical, eccentricity, inc_rad, omega_rad)